In [1]:
n_qubits = 3

## Step 1: Create Grover n-qubit, compile to L3, and save ISA (QPY)


In [2]:
from __future__ import annotations

from datetime import datetime
from pathlib import Path
import glob

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit import qpy

from ibm_quantum_connector import QuantumServiceManager
from grover_algorithm import grover_algorithm

# Check if circuit already exists in artifacts
artifacts = Path('artifacts')
artifacts.mkdir(exist_ok=True)

# Look for existing QPY files matching the pattern
existing_qpy_files = sorted(glob.glob(str(artifacts / f'grover{n_qubits}_l3_*.qpy')), reverse=True)

if existing_qpy_files:
    # Load existing circuit
    qpy_path = Path(existing_qpy_files[0])
    print(f'📁 Found existing circuit: {qpy_path.name}')
    with open(qpy_path, 'rb') as f:
        isa = list(qpy.load(f))[0]
    print('✅ Loaded existing ISA circuit')
else:
    # Generate new circuit
    print('🔧 No existing circuit found, generating new one...')
    
    # Connect and select backend
    svc = QuantumServiceManager(config_file='quantum_config.json')
    assert svc.connect(), 'Failed to connect to IBM Quantum service'
    backend = svc.select_backend()
    assert backend is not None, 'Failed to select backend'
    
    # Build source circuit and compile to L3
    src_qc = grover_algorithm(n_qubits)
    pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
    isa = pm.run(src_qc)
    
    # Save ISA QPY
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    qpy_path = artifacts / f'grover{n_qubits}_l3_{backend.name}_{ts}.qpy'
    with open(qpy_path, 'wb') as f:
        qpy.dump([isa], f)
    
    print(f'✅ Saved new ISA QPY to: {qpy_path}')
isa


📁 Found existing circuit: grover3_l3_ibm_fez_20251105_150131.qpy
✅ Loaded existing ISA circuit


In [ ]:
# Draw the L3 ISA circuit and save a PNG next to the QPY
import matplotlib.pyplot as plt
from pathlib import Path

try:
    fig = isa.draw(output='mpl')
    png_path = Path(str(qpy_path).replace('.qpy', '.png'))
    fig.savefig(png_path, dpi=180, bbox_inches='tight')
    print('Saved circuit PNG to:', png_path)
    plt.show()
except NameError as e:
    raise RuntimeError('Please run Step 1 cell first to create `isa` and `qpy_path`.') from e


In [ ]:
from qiskit import qpy
import glob

# Alternative: Load the latest QPY circuit manually (if not using Step 1 cell)
# This cell is optional if you've already run Step 1

qpy_files = sorted(glob.glob('artifacts/*.qpy'), reverse=True)
if not qpy_files:
    raise FileNotFoundError("No .qpy files found in the artifacts directory.")
qpy_path = qpy_files[0]

print(f'📁 Loading circuit: {qpy_path}')
with open(qpy_path, 'rb') as f:
    isa = list(qpy.load(f))[0]
print('✅ Circuit loaded')


In [ ]:
from quantum_executor import QuantumExecutor
qe = QuantumExecutor( config_file='quantum_config.json')

qe.run_circuit(isa_circuit=isa, shots=2048, execution_type='ideal_simulator')

{'success': True,
 'execution_type': 'ideal_simulator',
 'backend': 'ibm_fez',
 'job_id': None,
 'counts': {'000': 43,
  '001': 31,
  '010': 45,
  '011': 26,
  '100': 31,
  '101': 29,
  '110': 32,
  '111': 787},
 'probabilities': {'000': 0.0419921875,
  '001': 0.0302734375,
  '010': 0.0439453125,
  '011': 0.025390625,
  '100': 0.0302734375,
  '101': 0.0283203125,
  '110': 0.03125,
  '111': 0.7685546875},
 'shots': 1024,
 'method': 'Aer SamplerV2'}

In [ ]:
qe.run_circuit(isa_circuit=isa, shots=2048, execution_type='noisy_simulator')

In [ ]:
qe.run_circuit(isa_circuit=isa, shots=2048, execution_type='real_device')

## Step 2

In [25]:
from quantum_executor import QuantumExecutor
from delta_debug import run_delta_debug_on_isa

# Step 2: Run Delta Debugging on the compiled ISA circuit `isa`
# The debugger automatically detects logical qubits from measurements

qe = QuantumExecutor(config_file='quantum_config.json')
result = run_delta_debug_on_isa(
    executor=qe,
    isa_circuit=isa,
    tolerance=0.01,  # Default tolerance 
    max_granularity=16,  # Max splitting depth
    test_mode=False
)


In [19]:
import delta_debug_visualizer 
import importlib
importlib.reload(delta_debug_visualizer)

<module 'delta_debug_visualizer' from '/Users/qzydustin/WorkSpace/quantum/delta_debug_visualizer.py'>

In [26]:
# ========== Generate and display the HTML visualization report ==========
from delta_debug_visualizer import generate_html_report
from IPython.display import IFrame, display

# Generate the HTML report
html_path = generate_html_report(result)

# Display key information in English
print(f"\n📊 Key Findings:")
print(f"  • Total segments: {result['total_segments']}")
print(f"  • Problematic segments: {len(result['problematic_segments'])}")
print(f"  • Problematic segment IDs: {result['problematic_segments']}")
print(f"  • Baseline loss: {result['baseline_loss']:.4f}")
print(f"  • Test count: {result['test_count']}")

print(f"\n✅ HTML report generated: {html_path}")
print("👇 See the interactive report below")

# Display the report directly in the notebook
display(IFrame(html_path, width=1000, height=800))

✅ HTML report generated: delta_debug_report_20251112_000329.html

📊 Key Findings:
  • Total segments: 38
  • Problematic segments: 1
  • Problematic segment IDs: [16]
  • Baseline loss: 0.4385
  • Test count: 58

✅ HTML report generated: delta_debug_report_20251112_000329.html
👇 See the interactive report below
